In [77]:
from datasets import load_dataset, Audio 

csv_path = '/kaggle/input/technical-terms-dataset/data_technical_terms.csv'

In [78]:
import pandas as pd

In [79]:
df = pd.read_csv(csv_path)

# Update the column by adding a string to each value
df['audio_path'] = df['audio_path'].apply(lambda x: f"/kaggle/input/technical-terms-dataset/audio_output/{x}")

# Save the modified DataFrame back to a CSV file
df.to_csv('updated_file.csv', index=False)

In [80]:
import os.path
os.path.isfile('/kaggle/input/technical-terms-dataset/audio_output/audio_output/ad_10_20241019_162519.wav')

True

In [81]:
dataset = load_dataset("csv", data_files="./updated_file.csv")

dataset = dataset.cast_column("audio_path", Audio(sampling_rate=16e3))

Generating train split: 0 examples [00:00, ? examples/s]

In [82]:
dataset = dataset['train']

In [83]:
dataset = dataset.select(range(30))

In [84]:
import torch
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import soundfile as sf
from pesq import pesq
from scipy.io import wavfile
import numpy as np



In [ ]:
import numpy as np
import librosa
import torch
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
import soundfile as sf
from scipy.io import wavfile

lsd_scores = []
mos_scores = []

LSD_MIN = 0 
LSD_MAX = 10  

def log_spectral_distance(ref_audio, gen_audio, sr=16000):
    min_length = min(len(ref_audio), len(gen_audio))
    ref_audio = ref_audio[:min_length]
    gen_audio = gen_audio[:min_length]

    ref_spectrum = np.abs(librosa.stft(ref_audio))
    gen_spectrum = np.abs(librosa.stft(gen_audio))
    ref_log = np.log(ref_spectrum + 1e-10)  
    gen_log = np.log(gen_spectrum + 1e-10)
    lsd = np.mean(np.sqrt(np.mean((ref_log - gen_log) ** 2, axis=0)))
    return lsd

def normalize_lsd_to_mos(lsd_score, lsd_min, lsd_max):
    if lsd_score < lsd_min:
        return 5.0 
    elif lsd_score > lsd_max:
        return 1.0  
    else:
        return 5 - ((lsd_score - lsd_min) / (lsd_max - lsd_min) * 4)

x = 0
for sample in dataset:
    text = sample['sentence'] 
    original_audio_path = sample['audio_path']['path']  

 
    inputs = processor(text=text, return_tensors="pt")
    generated_speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

    generated_audio_path = "generated_speech.wav"
    sf.write(generated_audio_path, generated_speech.numpy(), samplerate=16000)
    print(f"Processing sample {x}")
    x += 1

    fs_ref, ref_audio = wavfile.read(original_audio_path)
    fs_deg, deg_audio = wavfile.read(generated_audio_path)

    if ref_audio.dtype == np.int16:
        ref_audio = ref_audio.astype(np.float32) / np.iinfo(np.int16).max  
    if deg_audio.dtype == np.int16:
        deg_audio = deg_audio.astype(np.float32) / np.iinfo(np.int16).max 

    if fs_ref == fs_deg == 16000:
        try:
            lsd_score = log_spectral_distance(ref_audio, deg_audio, sr=16000)
            lsd_scores.append(lsd_score)
#             print(f"LSD score: {lsd_score}")

            mos_score = normalize_lsd_to_mos(lsd_score, LSD_MIN, LSD_MAX)
            mos_scores.append(mos_score)
#             print(f"MOS score (from LSD): {mos_score}")

        except Exception as e:
            print(f"Error computing LSD for sample {sample['audio_path']['path']}: {e}")
    else:
        print(f"Sample rate mismatch for {sample['audio_path']['path']}")

average_lsd = np.mean(lsd_scores)
average_mos = np.mean(mos_scores)

print(f"Average Log Spectral Distance (LSD) score: {average_lsd}")
print(f"Average MOS score (from LSD): {average_mos}")


Processing sample 0
Processing sample 1
Processing sample 2
Processing sample 3
Processing sample 4
Processing sample 5
Processing sample 6
Processing sample 7
Processing sample 8
Processing sample 9
Processing sample 10
Processing sample 11
Processing sample 12
Processing sample 13
Processing sample 14
Processing sample 15
Processing sample 16
Processing sample 17
Processing sample 18
Processing sample 19
Processing sample 20
Processing sample 21
Processing sample 22
Processing sample 23
Processing sample 24
Processing sample 25
Processing sample 26
Processing sample 27
Processing sample 28
Processing sample 29
Average Log Spectral Distance (LSD) score: 2.0446481704711914
Average MOS score (from LSD): 4.1821407731374105


In [86]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import torch.nn.utils.prune as prune
import soundfile as sf
from datasets import load_dataset

def prune_model(model, amount=0.15):
    """
    Apply L1 unstructured pruning only to specific layers
    amount: percentage of weights to prune (0.15 = 15%)
    """
    for name, module in model.named_modules():
        # Only prune specific layers to maintain quality
        if isinstance(module, torch.nn.Linear) and ('encoder.feed_forward' in name):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')

def apply_selective_quantization(model):
    """
    Apply very light quantization only to specific layers
    that don't heavily impact audio quality
    """
    quantized_model = torch.quantization.quantize_dynamic(
        model,
        {torch.nn.Linear}, 
        dtype=torch.float16,  
        inplace=True
    )
    
    return quantized_model

# model = SpeechT5ForTextToSpeech.from_pretrained("aarishshahmohsin/final_technical_terms_t5_finetuned")

print("Applying light pruning...")
prune_model(model)

print("Applying minimal quantization...")
model = apply_selective_quantization(model)

Applying light pruning...
Applying minimal quantization...


In [88]:
import numpy as np
import librosa
import torch
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
import soundfile as sf
from scipy.io import wavfile

lsd_scores = []
mos_scores = []

LSD_MIN = 0 
LSD_MAX = 10  

def log_spectral_distance(ref_audio, gen_audio, sr=16000):
    min_length = min(len(ref_audio), len(gen_audio))
    ref_audio = ref_audio[:min_length]
    gen_audio = gen_audio[:min_length]

    ref_spectrum = np.abs(librosa.stft(ref_audio))
    gen_spectrum = np.abs(librosa.stft(gen_audio))
    ref_log = np.log(ref_spectrum + 1e-10)  
    gen_log = np.log(gen_spectrum + 1e-10)
    lsd = np.mean(np.sqrt(np.mean((ref_log - gen_log) ** 2, axis=0)))
    return lsd

def normalize_lsd_to_mos(lsd_score, lsd_min, lsd_max):
    if lsd_score < lsd_min:
        return 5.0 
    elif lsd_score > lsd_max:
        return 1.0  
    else:
        return 5 - ((lsd_score - lsd_min) / (lsd_max - lsd_min) * 4)

x = 0
for sample in dataset:
    text = sample['sentence'] 
    original_audio_path = sample['audio_path']['path']  

 
    inputs = processor(text=text, return_tensors="pt")
    generated_speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

    generated_audio_path = "generated_speech.wav"
    sf.write(generated_audio_path, generated_speech.numpy(), samplerate=16000)
    print(f"Processing sample {x}")
    x += 1

    fs_ref, ref_audio = wavfile.read(original_audio_path)
    fs_deg, deg_audio = wavfile.read(generated_audio_path)

    if ref_audio.dtype == np.int16:
        ref_audio = ref_audio.astype(np.float32) / np.iinfo(np.int16).max  
    if deg_audio.dtype == np.int16:
        deg_audio = deg_audio.astype(np.float32) / np.iinfo(np.int16).max 

    if fs_ref == fs_deg == 16000:
        try:
            lsd_score = log_spectral_distance(ref_audio, deg_audio, sr=16000)
            lsd_scores.append(lsd_score)
#             print(f"LSD score: {lsd_score}")

            mos_score = normalize_lsd_to_mos(lsd_score, LSD_MIN, LSD_MAX)
            mos_scores.append(mos_score)
#             print(f"MOS score (from LSD): {mos_score}")

        except Exception as e:
            print(f"Error computing LSD for sample {sample['audio_path']['path']}: {e}")
    else:
        print(f"Sample rate mismatch for {sample['audio_path']['path']}")

average_lsd = np.mean(lsd_scores)
average_mos = np.mean(mos_scores)

print(f"Average Log Spectral Distance (LSD) score: {average_lsd}")
print(f"Average MOS score (from LSD): {average_mos}")


Processing sample 0
Processing sample 1
Processing sample 2
Processing sample 3
Processing sample 4
Processing sample 5
Processing sample 6
Processing sample 7
Processing sample 8
Processing sample 9
Processing sample 10
Processing sample 11
Processing sample 12
Processing sample 13
Processing sample 14
Processing sample 15
Processing sample 16
Processing sample 17
Processing sample 18
Processing sample 19
Processing sample 20
Processing sample 21
Processing sample 22
Processing sample 23
Processing sample 24
Processing sample 25
Processing sample 26
Processing sample 27
Processing sample 28
Processing sample 29
Average Log Spectral Distance (LSD) score: 2.0446481704711914
Average MOS score (from LSD): 4.1821407731374105
